In [ ]:
# 🚀 ENHANCED BTC LSTM MODEL - MORE FEATURES
## Building on the successful foundation with volatility features + bidirectional LSTM

**Enhancements over the working model:**
- ✅ Added volatility features (Rolling std, ATR ratios, Bollinger Band width)
- ✅ Bidirectional LSTM layer for better sequence modeling
- ✅ L40S GPU optimization with memory management
- ✅ Maintains the successful backtesting framework
- ✅ CPU fallback if GPU fails

**Expected Improvements:**
- Current: 1.01% MAPE → Target: 0.7-0.85% MAPE
- Better directional accuracy
- Enhanced volatility prediction


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✅ Enhanced BTC Model Setup Complete! Adding volatility features + bidirectional LSTM...")


In [ ]:
## 1. Enhanced Model Class with GPU Configuration
This class handles L40S GPU optimization and contains all enhanced features.


In [ ]:
class EnhancedBTCModel:
    def __init__(self):
        self.configure_gpu()
        self.scaler = None
        self.model = None
        
    def configure_gpu(self):
        """Configure L40S GPU with memory optimization"""
        try:
            # Clear any existing session
            tf.keras.backend.clear_session()
            
            # Configure GPU if available
            gpus = tf.config.experimental.list_physical_devices('GPU')
            if gpus:
                try:
                    # Enable memory growth for L40S
                    for gpu in gpus:
                        tf.config.experimental.set_memory_growth(gpu, True)
                    
                    # Set conservative memory limit (32GB out of 48GB)
                    tf.config.experimental.set_virtual_device_configuration(
                        gpus[0],
                        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=32768)]
                    )
                    
                    print(f"✅ L40S GPU configured: {len(gpus)} GPU(s) with 32GB memory limit")
                    
                except RuntimeError as e:
                    print(f"⚠️ GPU configuration warning: {e}")
                    print("🔄 Falling back to CPU mode...")
            else:
                print("ℹ️ No GPU detected, using CPU mode")
                
        except Exception as e:
            print(f"⚠️ GPU setup error: {e}")
            print("🔄 Continuing with CPU...")


In [ ]:
## 2. Technical Indicator Functions
Calculate RSI and ATR indicators used in the enhanced features.


In [ ]:
# Add technical indicator methods to the EnhancedBTCModel class
def calculate_rsi(self, prices, window=14):
    """Calculate RSI indicator"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_atr(self, high, low, close, window=14):
    """Calculate Average True Range"""
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    
    true_range = np.maximum(high_low, np.maximum(high_close, low_close))
    return true_range.rolling(window=window).mean()

# Add methods to the class
EnhancedBTCModel.calculate_rsi = calculate_rsi
EnhancedBTCModel.calculate_atr = calculate_atr

print("✅ Technical indicator functions added")


In [ ]:
## 3. Enhanced Data Collection with Volatility Features
🚀 **NEW FEATURES ADDED:**
- Rolling volatility (1h and 4h windows)
- ATR ratios and percentiles
- Bollinger Band width and position
- Price momentum (multiple timeframes)
- Volume-price relationship indicators
- Volatility regime detection


In [ ]:
def get_enhanced_btc_data(self):
    """Fetch BTC data with enhanced volatility features"""
    print("📊 Fetching BTC data with enhanced features...")
    btc = yf.download("BTC-USD", period="1y", interval="1h")
    
    df = pd.DataFrame()
    
    # Basic OHLCV (original features)
    df['price'] = btc['Close']
    df['volume'] = btc['Volume'] 
    df['high'] = btc['High']
    df['low'] = btc['Low']
    
    # Original technical indicators
    df['sma_20'] = df['price'].rolling(20).mean()
    df['rsi'] = self.calculate_rsi(df['price'])
    
    # 🚀 NEW VOLATILITY FEATURES
    print("🔧 Adding volatility features...")
    
    # 1. Rolling Standard Deviation (Volatility)
    df['volatility_1h'] = df['price'].pct_change().rolling(24).std()  # 24h rolling vol
    df['volatility_4h'] = df['price'].pct_change().rolling(96).std()  # 4 day rolling vol
    
    # 2. ATR and ATR Ratios
    df['atr'] = self.calculate_atr(df['high'], df['low'], df['price'])
    df['atr_ratio'] = df['atr'] / df['price']  # Normalized ATR
    df['atr_percentile'] = df['atr'].rolling(168).rank(pct=True)  # ATR relative position
    
    # 3. Bollinger Bands Width and Position
    bb_window = 20
    bb_std = 2
    sma_bb = df['price'].rolling(bb_window).mean()
    bb_std_val = df['price'].rolling(bb_window).std()
    
    df['bb_upper'] = sma_bb + (bb_std_val * bb_std)
    df['bb_lower'] = sma_bb - (bb_std_val * bb_std)
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / sma_bb  # Normalized width
    df['bb_position'] = (df['price'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    
    # 4. Price Momentum Features
    df['price_momentum_1h'] = df['price'].pct_change(1)
    df['price_momentum_4h'] = df['price'].pct_change(4)
    df['price_momentum_24h'] = df['price'].pct_change(24)
    
    # 5. Volume-Price Features
    df['volume_sma'] = df['volume'].rolling(24).mean()
    df['volume_ratio'] = df['volume'] / df['volume_sma']
    df['volume_price_trend'] = (df['volume_ratio'] * df['price_momentum_1h']).rolling(12).mean()
    
    # 6. Volatility Regime Detection
    df['vol_regime'] = np.where(df['volatility_1h'] > df['volatility_1h'].rolling(168).quantile(0.7), 1, 0)
    
    # Remove NaN values
    df = df.dropna()
    
    print(f"✅ Enhanced dataset created: {len(df)} data points with {df.shape[1]} features")
    print(f"📊 Date range: {df.index[0]} to {df.index[-1]}")
    print(f"🔧 Features: {list(df.columns)}")
    
    return df

# Add method to the class
EnhancedBTCModel.get_enhanced_btc_data = get_enhanced_btc_data


In [ ]:
## 4. Enhanced Data Preparation
Time-aware data splitting to prevent data leakage and proper feature scaling.


In [ ]:
def prepare_enhanced_data(self, df, seq_length=60):
    """Prepare data with all enhanced features"""
    print("🔄 Preparing enhanced data...")
    
    # Scale all data
    self.scaler = MinMaxScaler()
    scaled_data = self.scaler.fit_transform(df)
    
    # Create sequences
    X, y = [], []
    for i in range(seq_length, len(scaled_data)):
        X.append(scaled_data[i-seq_length:i])
        y.append(scaled_data[i, 0])  # Predict price (first column)
    
    X, y = np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)
    
    # Time-aware split (preserve temporal order)
    split_idx = int(0.8 * len(X))
    val_split_idx = int(0.9 * len(X))
    
    X_train = X[:split_idx]
    X_val = X[split_idx:val_split_idx]
    X_test = X[val_split_idx:]
    
    y_train = y[:split_idx]
    y_val = y[split_idx:val_split_idx]
    y_test = y[val_split_idx:]
    
    print(f"✅ Data prepared:")
    print(f"   Training samples: {len(X_train)} | Features: {X_train.shape[2]}")
    print(f"   Validation samples: {len(X_val)}")
    print(f"   Test samples: {len(X_test)}")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Add method to the class
EnhancedBTCModel.prepare_enhanced_data = prepare_enhanced_data


In [ ]:
## 5. Enhanced Model Architecture with Bidirectional LSTM
🚀 **KEY IMPROVEMENTS:**
- Bidirectional LSTM in the middle layer for better sequence understanding
- BatchNormalization for training stability
- Enhanced optimizer configuration
- Memory-efficient parameter allocation


In [ ]:
def create_enhanced_model(self, input_shape):
    """Create enhanced model with bidirectional LSTM"""
    print("🏗️ Building enhanced LSTM model with bidirectional layer...")
    
    model = Sequential([
        # First LSTM layer (regular)
        LSTM(50, return_sequences=True, input_shape=input_shape, dropout=0.2),
        BatchNormalization(),
        
        # Bidirectional LSTM layer (enhanced sequence modeling)
        Bidirectional(LSTM(32, return_sequences=True, dropout=0.2)),
        BatchNormalization(),
        
        # Final LSTM layer (regular)
        LSTM(50, dropout=0.2),
        BatchNormalization(),
        
        # Dense layers
        Dense(25, activation='relu'),
        Dropout(0.3),
        Dense(1)
    ])
    
    # Enhanced optimizer
    optimizer = Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7
    )
    
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )
    
    print(f"✅ Enhanced model built with {model.count_params():,} parameters")
    print("🔧 Architecture: LSTM(50) → Bidirectional(LSTM(32)) → LSTM(50) → Dense")
    
    return model

# Add method to the class
EnhancedBTCModel.create_enhanced_model = create_enhanced_model


In [ ]:
## 6. Enhanced Training with Advanced Callbacks
Improved training strategy with better monitoring and learning rate scheduling.


In [ ]:
def train_enhanced_model(self, X_train, y_train, X_val, y_val, epochs=50):
    """Train with enhanced callbacks and monitoring"""
    print("🚀 Training enhanced model...")
    
    # Enhanced callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=8,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    # Conservative batch size for stability
    batch_size = 32
    
    try:
        print(f"🎯 Training with batch_size={batch_size}, {epochs} epochs max")
        
        history = self.model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )
        
        return history
        
    except Exception as e:
        print(f"❌ Training error: {e}")
        print("💡 Try reducing batch_size or sequence_length")
        return None

# Add method to the class
EnhancedBTCModel.train_enhanced_model = train_enhanced_model


In [ ]:
## 7. Enhanced Evaluation with Multiple Metrics
Comprehensive evaluation including directional accuracy and enhanced visualizations.


In [ ]:
def evaluate_enhanced_model(self, X_test, y_test, df):
    """Enhanced evaluation with multiple metrics"""
    print("📈 Evaluating enhanced model...")
    
    # Make predictions
    predictions = self.model.predict(X_test, verbose=0)
    
    # Convert back to real prices
    dummy_array = np.zeros((len(predictions), df.shape[1]))
    dummy_array[:, 0] = predictions.flatten()
    pred_prices = self.scaler.inverse_transform(dummy_array)[:, 0]
    
    dummy_array = np.zeros((len(y_test), df.shape[1]))
    dummy_array[:, 0] = y_test
    actual_prices = self.scaler.inverse_transform(dummy_array)[:, 0]
    
    # Calculate enhanced metrics
    mae = np.mean(np.abs(pred_prices - actual_prices))
    mape = np.mean(np.abs((actual_prices - pred_prices) / actual_prices)) * 100
    rmse = np.sqrt(np.mean((pred_prices - actual_prices) ** 2))
    
    # Directional accuracy
    actual_direction = np.sign(np.diff(actual_prices))
    pred_direction = np.sign(np.diff(pred_prices))
    directional_accuracy = np.mean(actual_direction == pred_direction) * 100
    
    print(f"💰 Enhanced Results:")
    print(f"Mean Absolute Error: ${mae:.2f}")
    print(f"Mean Absolute Percentage Error: {mape:.2f}%")
    print(f"Root Mean Square Error: ${rmse:.2f}")
    print(f"Directional Accuracy: {directional_accuracy:.1f}%")
    
    # Latest prediction
    current_price = actual_prices[-1]
    next_pred = pred_prices[-1]
    change = ((next_pred - current_price) / current_price) * 100
    
    print(f"\n🔮 Enhanced Prediction:")
    print(f"Current BTC Price: ${current_price:.2f}")
    print(f"Predicted Next Price: ${next_pred:.2f}")
    print(f"Predicted Change: {change:+.2f}%")
    
    return {
        'mae': mae,
        'mape': mape,
        'rmse': rmse,
        'directional_accuracy': directional_accuracy,
        'predictions': pred_prices,
        'actual': actual_prices
    }

# Add method to the class
EnhancedBTCModel.evaluate_enhanced_model = evaluate_enhanced_model


In [ ]:
## 8. Enhanced Visualization
Comprehensive plotting function for detailed analysis of model performance.


In [ ]:
def plot_enhanced_results(results):
    """Plot enhanced results with multiple visualizations"""
    pred_prices = results['predictions']
    actual_prices = results['actual']
    
    plt.figure(figsize=(15, 10))
    
    # Main prediction plot
    plt.subplot(2, 2, 1)
    plt.plot(actual_prices[-200:], label='Actual Price', color='blue', alpha=0.8)
    plt.plot(pred_prices[-200:], label='Predicted Price', color='red', alpha=0.8)
    plt.title('Enhanced BTC Price Prediction - Last 200 Hours')
    plt.xlabel('Hours')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Error distribution
    plt.subplot(2, 2, 2)
    errors = pred_prices - actual_prices
    plt.hist(errors, bins=50, alpha=0.7, color='orange')
    plt.title('Prediction Error Distribution')
    plt.xlabel('Error ($)')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    
    # Directional accuracy over time
    plt.subplot(2, 2, 3)
    actual_direction = np.sign(np.diff(actual_prices))
    pred_direction = np.sign(np.diff(pred_prices))
    
    window = 50
    rolling_dir_acc = []
    for i in range(window, len(actual_direction)):
        acc = np.mean(actual_direction[i-window:i] == pred_direction[i-window:i]) * 100
        rolling_dir_acc.append(acc)
    
    plt.plot(rolling_dir_acc)
    plt.title(f'Rolling Directional Accuracy ({window}h window)')
    plt.xlabel('Time')
    plt.ylabel('Accuracy (%)')
    plt.grid(True, alpha=0.3)
    
    # Scatter plot
    plt.subplot(2, 2, 4)
    plt.scatter(actual_prices, pred_prices, alpha=0.5, s=1)
    plt.plot([actual_prices.min(), actual_prices.max()], 
            [actual_prices.min(), actual_prices.max()], 'r--', lw=2)
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Actual Price ($)')
    plt.ylabel('Predicted Price ($)')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create function for plotting
plot_results = plot_enhanced_results


In [ ]:
## 9. Quick Test Function
Test enhanced features and show feature correlations before running the full pipeline.


In [ ]:
# Quick test of enhanced features
print("🧪 Quick test of enhanced model...")

model = EnhancedBTCModel()

# Test data fetching
try:
    df = model.get_enhanced_btc_data()
    print(f"✅ Data test passed: {df.shape}")
    
    # Show feature correlation
    feature_cols = [col for col in df.columns if col != 'price']
    correlation_with_price = df[feature_cols].corrwith(df['price']).abs().sort_values(ascending=False)
    
    print(f"\n📊 Top 10 most correlated features with price:")
    for feature, corr in correlation_with_price.head(10).items():
        print(f"   {feature}: {corr:.3f}")
        
    print("\n✅ Quick test completed successfully!")
    
except Exception as e:
    print(f"❌ Test failed: {e}")
    print("💡 Check data connectivity and dependencies")


In [ ]:
## 10. Run Complete Enhanced Pipeline
Execute the full enhanced pipeline with all improvements.


In [ ]:
print("🚀 Starting Enhanced BTC LSTM Pipeline")
print("=" * 60)

# Create enhanced model instance
enhanced_model = EnhancedBTCModel()

try:
    # Step 1: Get enhanced data
    print("\n📊 Step 1: Fetching enhanced data...")
    df = enhanced_model.get_enhanced_btc_data()
    
    # Step 2: Prepare data
    print("\n🔄 Step 2: Preparing enhanced data...")
    X_train, X_val, X_test, y_train, y_val, y_test = enhanced_model.prepare_enhanced_data(df)
    
    # Step 3: Create enhanced model
    print("\n🏗️ Step 3: Building enhanced model...")
    enhanced_model.model = enhanced_model.create_enhanced_model((X_train.shape[1], X_train.shape[2]))
    
    # Step 4: Train model
    print("\n🚀 Step 4: Training enhanced model...")
    history = enhanced_model.train_enhanced_model(X_train, y_train, X_val, y_val)
    
    if history is None:
        print("❌ Training failed")
    else:
        # Step 5: Evaluate model
        print("\n📈 Step 5: Evaluating enhanced model...")
        results = enhanced_model.evaluate_enhanced_model(X_test, y_test, df)
        
        print(f"\n🎉 Enhanced model training complete!")
        print(f"📊 Improvement metrics:")
        print(f"   MAPE: {results['mape']:.2f}% (target: <0.85%)")
        print(f"   Directional Accuracy: {results['directional_accuracy']:.1f}%")
        
        # Store results for plotting
        enhanced_results = results
        enhanced_data = df
        enhanced_history = history
        
        print("\n✅ Enhanced pipeline completed successfully!")
        
except Exception as e:
    print(f"❌ Pipeline error: {e}")
    print("💡 Check GPU memory and data availability")


In [ ]:
## 11. Plot Enhanced Results
Visualize the enhanced model performance with detailed charts.


In [ ]:
# Plot enhanced results if training was successful
try:
    if 'enhanced_results' in locals():
        print("📊 Plotting enhanced results...")
        plot_results(enhanced_results)
        
        # Plot training history
        if 'enhanced_history' in locals():
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 2, 1)
            plt.plot(enhanced_history.history['loss'], label='Training Loss')
            plt.plot(enhanced_history.history['val_loss'], label='Validation Loss')
            plt.title('Enhanced Model Training Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.subplot(1, 2, 2)
            plt.plot(enhanced_history.history['mae'], label='Training MAE')
            plt.plot(enhanced_history.history['val_mae'], label='Validation MAE')
            plt.title('Enhanced Model Training MAE')
            plt.xlabel('Epoch')
            plt.ylabel('MAE')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
        print("✅ Visualization complete!")
    else:
        print("⚠️ No results to plot - run the enhanced pipeline first")
        
except NameError:
    print("⚠️ No results available - run the enhanced pipeline first")
except Exception as e:
    print(f"❌ Plotting error: {e}")


In [ ]:
## 🎯 Enhanced Model Summary

**Key Improvements Over Original Model:**
- ✅ **12 New Volatility Features**: Rolling volatility, ATR ratios, Bollinger Bands, momentum indicators
- ✅ **Bidirectional LSTM**: Enhanced sequence understanding with backward temporal dependencies  
- ✅ **L40S GPU Optimization**: Memory management for 32GB allocation with fallback to CPU
- ✅ **Advanced Training**: Learning rate scheduling, early stopping, batch normalization
- ✅ **Enhanced Evaluation**: Directional accuracy, multiple metrics, comprehensive visualizations

**Expected Performance:**
- Target MAPE: 0.7-0.85% (vs original 1.01%)
- Enhanced directional accuracy
- Better volatility prediction
- More stable training with GPU optimization

**Memory Management:**
- Conservative 32GB GPU limit (out of 48GB available)
- Float32 precision for memory efficiency
- Batch size optimization for stability
- CPU fallback if GPU fails
